In [1]:
# Yusuf Günay
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
#excel dosyasını okur ve sütun isimlerini belirtir
data = pd.read_excel("World of Tanks - tanks.gg.xlsx", names=["User", "Tank_Name", "Tank_Level", "Tank_Type", "Tank_Nation", "Battle_Count"])
print(data)

              User     Tank_Name Tank_Level Tank_Type Tank_Nation  \
0      Altks13\nEU          IS-4          X        HT        USSR   
1      Altks13\nEU       T-34-85         VI        MT        USSR   
2      Altks13\nEU         E 100          X        HT     Germany   
3      Altks13\nEU          T-54         IX        MT        USSR   
4      Altks13\nEU      AMX 50 B          X        HT      France   
...            ...           ...        ...       ...         ...   
39995  Altks13\nEU         KV-85         VI        HT        USSR   
39996  Altks13\nEU          KV-1          V        HT        USSR   
39997  Altks13\nEU        T110E4          X        TD         USA   
39998  Altks13\nEU  Char Futur 4         IX        MT      France   
39999  Altks13\nEU          IS-3       VIII        HT        USSR   

       Battle_Count  
0             1.603  
1             1.483  
2             1.396  
3             1.287  
4             1.222  
...             ...  
39995         5.0

In [3]:
number_of_tanks  = data.groupby('Tank_Name').size().sort_values(ascending = False)
print(number_of_tanks)

Tank_Name
IS-3            631
KV-1            449
IS              425
T-10            384
KV-85           366
               ... 
BZ-72-1           1
Pz. 58            1
Chaffee 594       1
T-26G FT          1
Type 4 Heavy      1
Length: 770, dtype: int64


In [4]:
new_data = []

for username in data['User'].unique():
    Tank_Names = data[data['User'] == username]['Tank_Name'].tolist()
    new_data.append({'User': username, 'Tanks': Tank_Names})

data = pd.DataFrame(new_data)

In [5]:
data = pd.DataFrame(new_data)
data

,User,Tanks
0,Altks13\nEU,"[IS-4, T-34-85, E 100, T-54, AMX 50 B, KV-2, A..."
1,Lux54\nEU,"[G.Pz. Mk. VI, Medium I, L.Tr., MS-1, Tetrarch]"
2,SiggeSur\nEU,"[UDES 03, UDES 14 5, Strv 103-0, Guard, Rhm.-B..."
3,Zwer_314\nEU,"[L.Tr., M3 Stuart, M5 Stuart, Chaffee, T37, Pz..."
4,_BelayaSmert_\nEU,"[AMX 13 90, AMX 30, Foch 155, AMX ELC bis, Def..."
...,...,...
1657,P_E_T_R_O_74\nEU,"[E 75, KV-2, G.W. Panther, O-I, SU-100, Hetzer..."
1658,Aleksandr_Djomin\nEU,"[Super Hellcat, IS-3, Pz. IV F2, KV-1SA, IS, S..."
1659,Virtus_Vindex\nEU,"[Hummel, M44, FV3805, G.W. Tiger P, SU-14-2, F..."
1660,skipeur_2\nEU,"[O-I Exp., G.W. Panther, BT-7 art., T28 Prot.,..."


In [6]:
def custom_tokenizer(x):
    return x.split()

# TfidfVectorizer'ı oluşturur
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)

# Tanks sütununu kullanarak TF-IDF matrisine dönüştürür
tfidf_matrix = vectorizer.fit_transform(data['Tanks'].astype(str)).toarray()

# Sütun isimlerini alır
feature_names = vectorizer.get_feature_names_out()

# TF-IDF matrisini bir DataFrame'e dönüştürür
data = pd.DataFrame(tfidf_matrix, columns=feature_names)

# TF-IDF matrisini ikili formata dönüştürür
data_bin = data > 0

In [7]:
# Frequent Itemsets oluşturur
frequent_itemsets = apriori(data_bin, min_support=0.1, use_colnames=True)


# Support'a göre sıralar
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

print(frequent_itemsets)

      support               itemsets
0    0.507220                 ('amx)
24   0.432010                ('obj.)
46   0.383875                  ('vk)
26   0.367629                 ('pz.)
13   0.359206              ('is-3',)
..        ...                    ...
296  0.101083        (t, 'b-c, 'amx)
223  0.100481            (t',, 'pz.)
304  0.100481  (90',, 'amx, 'is-3',)
320  0.100481        (t',, 'b-c, 12)
236  0.100481           (iv, 'tiger)

[380 rows x 2 columns]


In [8]:
assoc_rules = association_rules(frequent_itemsets)

# Association Rules'ları görüntüler
assoc_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(13),('amx),0.278580,0.507220,0.276173,0.991361,1.954498,0.134872,57.039260,0.676942
1,('škoda),(t),0.200361,0.336943,0.200361,1.000000,2.967857,0.132851,inf,0.829195
2,"('t-10',)","('is-3',)",0.220818,0.359206,0.178700,0.809264,2.252927,0.099381,3.359593,0.713740
3,"(13, 90',)",('amx),0.176294,0.507220,0.174489,0.989761,1.951344,0.085069,48.128159,0.591877
4,"(90',, 'amx)",(13),0.217208,0.278580,0.174489,0.803324,2.883639,0.113979,3.668065,0.834470
...,...,...,...,...,...,...,...,...,...,...
166,"(h',, 'tiger, 'pz.)",('vk),0.115523,0.383875,0.101083,0.875000,2.279389,0.056736,4.929001,0.634597
167,"('tiger, 'vk, 'pz.)","(h',)",0.117930,0.279783,0.101083,0.857143,3.063594,0.068088,5.041516,0.763643
168,"('is',, 'obj.)","('is-3',)",0.125752,0.359206,0.101083,0.803828,2.237792,0.055912,3.266488,0.632693
169,"(90',, 'is-3',)",('amx),0.121540,0.507220,0.100481,0.826733,1.629928,0.038834,2.844043,0.439948


In [9]:
frequent_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(len) > 1]
frequent_itemsets['TF IDF Scores'] = frequent_itemsets['itemsets'].apply(
    lambda x: data[x].sum().sum())

In [10]:
frequent_itemsets.sort_values(by = 'TF IDF Scores' , ascending = False)[['itemsets' , 'TF IDF Scores']]

,itemsets,TF IDF Scores
375,"(h',, 'tiger, 'vk, 'pz.)",287.568137
371,"(13, 'b-c, 'amx, t',)",272.903532
372,"(13, 90',, 'amx, t',)",267.003597
378,"(ii',, i',, 'tiger, 'vk)",266.952469
377,"(ii',, h',, 'vk, 'tiger)",265.944459
...,...,...
144,"('char, futur)",52.307596
143,"('char, 4',)",52.135561
270,"(4',, futur)",51.556652
228,"(hellcat',, 'super)",51.268707
